In [8]:
import pandas as pd
from collections import defaultdict
import warnings
import visualize
import neat
import multiprocessing
import os
import matplotlib.pyplot as plt

import numpy as np

In [9]:
necoc = 26
generations = int(3000/necoc)
generations

115

In [10]:
df = pd.read_csv('letter-recognition.data',header=None)

train_df = df[:1600]
test_df = df[16000:16000+400]

X = train_df.ix[:,1:16].to_numpy() 
y = np.array([ord(l)-ord('A') for l in train_df.ix[:,0].to_numpy()])

testX = test_df.ix[:,1:16].to_numpy() 
testy = np.array([ord(l)-ord('A') for l in test_df.ix[:,0].to_numpy()])

In [11]:
def softmax(x, axis=1):
    # 计算每行的最大值
    row_max = x.max(axis=axis)
 
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

In [12]:
def get_winner(class1):
    
    numbers = [*range(26)] # number 2 > number 1
    number_of_classification = len(numbers)
    #number_of_sampling = 100

    list_y = y.tolist()
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

        
    class2 = []

    class1_indexes = []
    class2_indexes = []

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)

    print("number of class1:{}".format(len(class1_label)))
    print("number of class2:{}".format(len(class2_label)))

    samplesize = 1600
    x_inputs =[tuple(c) for c in X[class1_indexes].tolist()] + [tuple(c) for c in X[class2_indexes].tolist()]
    x_outputs = [tuple([c]) for c in class1_label + class2_label]
    
    
    def eval_genomes(genomes, config):
        for genome_id, genome in genomes:
            net = neat.nn.FeedForwardNetwork.create(genome, config)

            outputs = []
            for xi in x_inputs:
                output = net.activate(xi)
                outputs.append(output)


            px_outputs = softmax(np.array(outputs).reshape(samplesize, 2), axis=1)
            # the index of maximum in each line
            pred_outputs = np.argmax(px_outputs, axis = 1)
            real_outputs = np.array(x_outputs).reshape(samplesize,)

            acc = np.sum(pred_outputs == real_outputs)/samplesize

            genome.fitness = acc

    def run(config_file):
        # Load configuration.
        config = neat.Config(
            neat.DefaultGenome,
            neat.DefaultReproduction,
            neat.DefaultSpeciesSet,
            neat.DefaultStagnation,
            config_file,
        )

        # Create the population, which is the top-level object for a NEAT run.
        p = neat.Population(config)

        # add a stdout reporter to show progress in the terminal
        reporter = neat.StdOutReporter(False)
        p.add_reporter(reporter)
        stats = neat.StatisticsReporter()
        p.add_reporter(stats)
        #checkpointer = neat.Checkpointer(100)
        #p.add_reporter(checkpointer)
        # Run for up to 300 generations.
        winner = p.run(eval_genomes, generations)

        return [stats, winner]
    
    local_dir = os.getcwd()
    config_path = os.path.join(local_dir, "config-feedforward-letter2")
    [stats, winner] = run(config_path)
    
    return winner

In [13]:
#class1 = [0, 2, 4, 6, 8]
#winner1 = get_winner(class1)

In [14]:
winner_list = []
classes = [[0],[1],[2],[3],[4],[5],[6],[7],[8],[9],[10],[11],[12],[13],[14],[15],[16],[17],[18],[19],[20],[21],[22],[23],[24],[25]]

for i,class1 in enumerate(classes):
    print("running:{}/26".format(i))
    winner = get_winner(class1)
    winner_list.append(winner)

running:0/30
number of class1:64
number of class2:1536

 ****** Running generation 0 ****** 

Population's average fitness: 0.53300 stdev: 0.39456
Best fitness: 0.96125 - size: (2, 3) - species 6 - id 87
Average adjusted fitness: 0.490
Mean genetic distance 2.793, standard deviation 0.511
Population of 199 members in 23 species
Total extinctions: 0
Generation time: 3.006 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.78247 stdev: 0.32229
Best fitness: 0.96500 - size: (3, 4) - species 1 - id 255
Average adjusted fitness: 0.693
Mean genetic distance 2.928, standard deviation 0.516
Population of 202 members in 24 species
Total extinctions: 0
Generation time: 3.301 sec (3.154 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.78945 stdev: 0.32784
Best fitness: 0.96500 - size: (3, 4) - species 1 - id 255
Average adjusted fitness: 0.679
Mean genetic distance 2.920, standard deviation 0.534
Population of 205 members in 27 species
Total 

Population's average fitness: 0.78924 stdev: 0.33065
Best fitness: 0.98813 - size: (7, 14) - species 1 - id 2781
Average adjusted fitness: 0.757
Mean genetic distance 2.987, standard deviation 0.636
Population of 202 members in 12 species
Total extinctions: 0
Generation time: 4.622 sec (4.450 average)

 ****** Running generation 25 ****** 

Population's average fitness: 0.80402 stdev: 0.31611
Best fitness: 0.98813 - size: (10, 17) - species 4 - id 3862
Average adjusted fitness: 0.772
Mean genetic distance 2.925, standard deviation 0.629
Population of 201 members in 13 species
Total extinctions: 0
Generation time: 4.893 sec (4.345 average)

 ****** Running generation 26 ****** 

Population's average fitness: 0.81554 stdev: 0.31146
Best fitness: 0.98813 - size: (10, 17) - species 4 - id 3862
Average adjusted fitness: 0.794
Mean genetic distance 2.933, standard deviation 0.616
Population of 200 members in 13 species
Total extinctions: 0
Generation time: 6.017 sec (4.532 average)

 ****** 

Generation time: 5.703 sec (5.493 average)

 ****** Running generation 48 ****** 

Population's average fitness: 0.76787 stdev: 0.34454
Best fitness: 0.99250 - size: (8, 16) - species 1 - id 6396
Average adjusted fitness: 0.720
Mean genetic distance 3.128, standard deviation 0.573
Population of 199 members in 19 species
Total extinctions: 0
Generation time: 5.926 sec (5.557 average)

 ****** Running generation 49 ****** 

Population's average fitness: 0.82015 stdev: 0.30960
Best fitness: 0.99250 - size: (8, 16) - species 1 - id 6396
Average adjusted fitness: 0.776
Mean genetic distance 3.116, standard deviation 0.564
Population of 198 members in 20 species
Total extinctions: 0
Generation time: 5.551 sec (5.591 average)

 ****** Running generation 50 ****** 

Population's average fitness: 0.80188 stdev: 0.32146
Best fitness: 0.99250 - size: (8, 16) - species 1 - id 6396
Average adjusted fitness: 0.741
Mean genetic distance 3.115, standard deviation 0.565
Population of 196 members in 20 

Population's average fitness: 0.85787 stdev: 0.24783
Best fitness: 0.99375 - size: (10, 17) - species 61 - id 11820
Average adjusted fitness: 0.805
Mean genetic distance 3.234, standard deviation 0.559
Population of 201 members in 13 species
Total extinctions: 0
Generation time: 6.719 sec (6.773 average)

 ****** Running generation 73 ****** 

Population's average fitness: 0.85429 stdev: 0.26723
Best fitness: 0.99375 - size: (10, 17) - species 69 - id 11820
Average adjusted fitness: 0.819
Mean genetic distance 3.252, standard deviation 0.563
Population of 198 members in 15 species
Total extinctions: 0
Generation time: 7.357 sec (6.833 average)

 ****** Running generation 74 ****** 

Population's average fitness: 0.79312 stdev: 0.33285
Best fitness: 0.99375 - size: (10, 17) - species 69 - id 11820
Average adjusted fitness: 0.725
Mean genetic distance 3.280, standard deviation 0.562
Population of 203 members in 15 species
Total extinctions: 0
Generation time: 7.412 sec (6.940 average)

 

Population's average fitness: 0.84216 stdev: 0.28144
Best fitness: 0.99625 - size: (12, 26) - species 68 - id 12493
Average adjusted fitness: 0.797
Mean genetic distance 3.199, standard deviation 0.570
Population of 198 members in 16 species
Total extinctions: 0
Generation time: 8.087 sec (8.363 average)

 ****** Running generation 97 ****** 

Population's average fitness: 0.83268 stdev: 0.29321
Best fitness: 0.99625 - size: (12, 26) - species 68 - id 12493
Average adjusted fitness: 0.795
Mean genetic distance 3.199, standard deviation 0.592
Population of 198 members in 15 species
Total extinctions: 0
Generation time: 6.968 sec (8.317 average)

 ****** Running generation 98 ****** 

Population's average fitness: 0.84737 stdev: 0.25890
Best fitness: 0.99625 - size: (12, 26) - species 68 - id 12493
Average adjusted fitness: 0.804
Mean genetic distance 3.191, standard deviation 0.590
Population of 202 members in 15 species
Total extinctions: 0
Generation time: 7.142 sec (8.294 average)

 

...

Population's average fitness: 0.83282 stdev: 0.29404
Best fitness: 0.97000 - size: (5, 8) - species 9 - id 1700
Average adjusted fitness: 0.785
Mean genetic distance 2.745, standard deviation 0.528
Population of 197 members in 17 species
Total extinctions: 0
Generation time: 3.115 sec (3.202 average)

 ****** Running generation 13 ****** 

Population's average fitness: 0.85366 stdev: 0.27306
Best fitness: 0.97000 - size: (5, 8) - species 9 - id 1700
Average adjusted fitness: 0.841
Mean genetic distance 2.748, standard deviation 0.545
Population of 200 members in 17 species
Total extinctions: 0
Generation time: 3.200 sec (3.206 average)

 ****** Running generation 14 ****** 

Population's average fitness: 0.82439 stdev: 0.30348
Best fitness: 0.97000 - size: (5, 8) - species 9 - id 1700
Average adjusted fitness: 0.806
Mean genetic distance 2.753, standard deviation 0.562
Population of 200 members in 18 species
Total extinctions: 0
Generation time: 3.420 sec (3.237 average)

 ****** Runni

Population's average fitness: 0.87489 stdev: 0.26402
Best fitness: 0.97062 - size: (6, 12) - species 13 - id 3396
Average adjusted fitness: 0.846
Mean genetic distance 3.005, standard deviation 0.550
Population of 206 members in 24 species
Total extinctions: 0
Generation time: 3.874 sec (3.869 average)

 ****** Running generation 37 ****** 

Population's average fitness: 0.85733 stdev: 0.28266
Best fitness: 0.97062 - size: (6, 12) - species 13 - id 3396
Average adjusted fitness: 0.826
Mean genetic distance 2.974, standard deviation 0.549
Population of 206 members in 20 species
Total extinctions: 0
Generation time: 4.034 sec (3.890 average)

 ****** Running generation 38 ****** 

Population's average fitness: 0.85668 stdev: 0.26924
Best fitness: 0.97062 - size: (6, 12) - species 13 - id 3396
Average adjusted fitness: 0.821
Mean genetic distance 2.913, standard deviation 0.579
Population of 198 members in 15 species
Total extinctions: 0
Generation time: 4.132 sec (3.931 average)

 ******

Population's average fitness: 0.83071 stdev: 0.29853
Best fitness: 0.97125 - size: (11, 22) - species 29 - id 8508
Average adjusted fitness: 0.796
Mean genetic distance 3.064, standard deviation 0.625
Population of 204 members in 13 species
Total extinctions: 0
Generation time: 4.040 sec (4.117 average)

 ****** Running generation 61 ****** 

Population's average fitness: 0.83982 stdev: 0.28619
Best fitness: 0.97188 - size: (7, 14) - species 13 - id 10156
Average adjusted fitness: 0.831
Mean genetic distance 3.076, standard deviation 0.633
Population of 199 members in 12 species
Total extinctions: 0
Generation time: 4.495 sec (4.166 average)

 ****** Running generation 62 ****** 

Population's average fitness: 0.83946 stdev: 0.28248
Best fitness: 0.97250 - size: (11, 19) - species 29 - id 10336
Average adjusted fitness: 0.817
Mean genetic distance 3.092, standard deviation 0.616
Population of 199 members in 13 species
Total extinctions: 0
Generation time: 4.111 sec (4.176 average)

 **

Population's average fitness: 0.88287 stdev: 0.25318
Best fitness: 0.97313 - size: (11, 19) - species 29 - id 12669
Average adjusted fitness: 0.855
Mean genetic distance 3.071, standard deviation 0.570
Population of 201 members in 15 species
Total extinctions: 0
Generation time: 4.911 sec (4.729 average)

 ****** Running generation 85 ****** 

Population's average fitness: 0.87383 stdev: 0.26186
Best fitness: 0.97313 - size: (11, 19) - species 29 - id 12669
Average adjusted fitness: 0.842
Mean genetic distance 3.087, standard deviation 0.574
Population of 199 members in 15 species
Total extinctions: 0
Generation time: 5.101 sec (4.780 average)

 ****** Running generation 86 ****** 

Population's average fitness: 0.88410 stdev: 0.24799
Best fitness: 0.97313 - size: (11, 19) - species 29 - id 12669
Average adjusted fitness: 0.852
Mean genetic distance 3.096, standard deviation 0.582
Population of 204 members in 14 species
Total extinctions: 0
Generation time: 5.090 sec (4.827 average)

 

Total extinctions: 0
Generation time: 5.259 sec (5.124 average)

 ****** Running generation 108 ****** 

Population's average fitness: 0.89709 stdev: 0.21963
Best fitness: 0.97375 - size: (13, 22) - species 29 - id 16887
Average adjusted fitness: 0.867
Mean genetic distance 3.040, standard deviation 0.555
Population of 198 members in 15 species
Total extinctions: 0
Generation time: 4.771 sec (5.073 average)

 ****** Running generation 109 ****** 

Population's average fitness: 0.87095 stdev: 0.26219
Best fitness: 0.97375 - size: (13, 22) - species 29 - id 16887
Average adjusted fitness: 0.840
Mean genetic distance 3.037, standard deviation 0.553
Population of 196 members in 15 species
Total extinctions: 0
Generation time: 4.817 sec (5.038 average)

 ****** Running generation 110 ****** 

Population's average fitness: 0.86316 stdev: 0.27705
Best fitness: 0.97375 - size: (13, 22) - species 29 - id 16887
Average adjusted fitness: 0.835
Mean genetic distance 3.027, standard deviation 0.571

In [17]:
local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-letter2")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

def get_pred_real(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = testX[j]
    test_y = testy[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

from random import randint
pred_value = []
error_list = []

for j in range(400):
    pred = []
    for i in range(26):
        [pred_outputs, real_outputs] = get_pred_real(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    pred_array = np.array(pred).reshape(26,)
    #print(pred_array)
    if (np.sum(pred_array) >= 1):
        pred_value.append(np.where(pred_array==1))
    else:
        pred_value.append(np.array(range(26)))

import random

list_P = []
for i in range(400):
    if (len(pred_value[i])== 1):
        #print(pred_value[i][0], digits.target[1617+i])
        list_P.append(random.sample(pred_value[i][0].tolist(),1)[0])
    else:
        list_P.append(random.sample(pred_value[i].tolist(),1)[0])
        
np.sum(list_P == testy)/400

0.27

In [18]:
def get_pred_train(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = X[j]
    test_y = y[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

pred_value = []
error_list = []

for j in range(1600):
    pred = []
    for i in range(26):
        [pred_outputs, real_outputs] = get_pred_train(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    pred_array = np.array(pred).reshape(26,)
    #print(pred_array)
    if (np.sum(pred_array) >= 1):
        pred_value.append(np.where(pred_array==1))
    else:
        pred_value.append(np.array(range(26)))

import random

list_P = []
for i in range(1600):
    if (len(pred_value[i])== 1):
        #print(pred_value[i][0], digits.target[1617+i])
        list_P.append(random.sample(pred_value[i][0].tolist(),1)[0])
    else:
        list_P.append(random.sample(pred_value[i].tolist(),1)[0])
print("Trainset acc:{}".format(np.sum(list_P == y)/1600))
winner_fitness = []
for winner in winner_list:
    #print(winner.fitness)
    winner_fitness.append(winner.fitness)
print("Avg Base Learner:{}".format(np.mean(winner_fitness)))

Trainset acc:0.308125
Avg Base Learner:0.9710576923076923


In [20]:
# program to check if there is exist a path between two vertices 
# of a graph 

from collections import defaultdict 

#This class represents a directed graph using adjacency list representation 
class Graph: 

	def __init__(self,vertices): 
		self.V= vertices #No. of vertices 
		self.graph = defaultdict(list) # default dictionary to store graph 

	# function to add an edge to graph 
	def addEdge(self,u,v): 
		self.graph[u].append(v) 
	
	# Use BFS to check path between s and d 
	def isReachable(self, s, d): 
		# Mark all the vertices as not visited 
		visited =[False]*(self.V) 

		# Create a queue for BFS 
		queue=[] 

		# Mark the source node as visited and enqueue it 
		queue.append(s) 
		visited[s] = True

		while queue: 

			#Dequeue a vertex from queue 
			n = queue.pop(0) 
			
			# If this adjacent node is the destination node, 
			# then return true 
			if n == d: 
				return True

			# Else, continue to do BFS 
			for i in self.graph[n]: 
				if visited[i] == False: 
					queue.append(i) 
					visited[i] = True
		# If BFS is complete without visited d 
		return False
    
    
def findAllPath(graph,start,end,path=[]):
    path = path +[start]
    if start == end:
        return [path]
 
    paths = [] #存储所有路径    
    for node in graph[start]:
        if node not in path:
            newpaths = findAllPath(graph,node,end,path) 
            for newpath in newpaths:
                paths.append(newpath)
    return paths

numbers = [*range(2)]
number_of_classification = len(numbers)

list_nodes_number = []
list_connection_number = []

for winner in winner_list:
    used_nodes = list(winner.nodes.keys())
    # create a dict for mapping 
    l1 = numbers + config.genome_config.input_keys + used_nodes[number_of_classification:]
    l2 = range(len(l1))
    dict_nodes = dict(zip(l1, l2))

    ### 将节点加入图
    g = Graph(len(l1))# inputs + outputs + used  
    # add connections in the graph
    for cg in winner.connections.values():
        if cg.enabled:
            g.addEdge(dict_nodes[cg.key[0]],dict_nodes[cg.key[1]])

    v = numbers
    list_connections = []
    for vi in v:
        for u in range(number_of_classification, len(config.genome_config.input_keys) + number_of_classification):
            list_connections.append(g.isReachable(u, vi))

    nodes_inputs = range(number_of_classification, number_of_classification+ 64)
    nodes_outputs = range(number_of_classification)
    nodes_mid = range(number_of_classification + 64, len(dict_nodes))

    # 找到所有输入和输出连接的路
    all_path = []
    for u in nodes_inputs:
        for v in nodes_outputs:
            path = findAllPath(g.graph, u, v)
            if path:
                all_path = all_path + path

    # 得到最长路，以及每个节点在哪个层级
    max_length = max([len(x) for x in all_path])
    nodes_tuples_list = []
    for path in all_path:
        #print(path)
        for node in path:
            nodes_tuples_list.append([node, path.index(node)])

    # 确定节点的层级（消除重复）       
    nodes_tuples_fixed_list = []
    for index in range(max_length):
        for nodes in nodes_tuples_list:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                if nodes not in nodes_tuples_fixed_list:
                    nodes_tuples_fixed_list.append(nodes)


    # 所有有连接的节点
    all_used_nodes = []
    for node in nodes_tuples_fixed_list:
        if node[0] not in all_used_nodes:
            all_used_nodes.append(node[0])

    # 确定节点的层级（最终确定）
    nodes_tuples_fixed_list_final = []

    for node in all_used_nodes:
        if node in nodes_inputs:
            nodes_tuples_fixed_list_final.append([node, 0])
        elif node in nodes_outputs:
            nodes_tuples_fixed_list_final.append([node, max_length - 1])
        else:
            list_node = []
            for nodes in nodes_tuples_fixed_list:
                if node == nodes[0]:
                    list_node.append(nodes)
            layer = 0
            for ele in list_node:
                # 删除重复项，取最大
                if ele[1] > layer:
                    layer = ele[1]
            nodes_tuples_fixed_list_final.append([node,layer])


    # 每个层级的节点合数
    length_of_layers = []
    number_of_nodes = 0
    for index in range(max_length):
        for nodes in nodes_tuples_fixed_list_final:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                number_of_nodes += 1
        length_of_layers.append(number_of_nodes)

    length_of_each_layer = []
    length_of_each_layer.append(length_of_layers[0])
    for i in range(1, max_length):
        length_of_each_layer.append(length_of_layers[i] - length_of_layers[i-1])

    # 输出每个层级的节点个数
    #print("length of each layers:", length_of_each_layer)

    # 所有端到端的路
    all_path_side2side = []
    for path in all_path:
        if len(path) == 2:
            all_path_side2side.append(path)
        else:
            for i in range(len(path)-1):
                all_path_side2side.append([path[i],path[i+1]])

    # 定义节点到绘图的字典
    dict_nodes_graph = dict()
    count_number_layer = [0] * max_length
    for nodes in nodes_tuples_fixed_list_final:
        node, layer = nodes
        dict_nodes_graph[node] = [count_number_layer[layer], layer]
        count_number_layer[layer] += 1
    list_nodes_number.append(np.sum(length_of_each_layer))
    list_connection_number.append(len(all_path_side2side))
    #print("Number of nodes:{} Number of connections:{}".format(np.sum(length_of_each_layer),len(all_path_side2side)))
print("Total nodes:{} Total connections::{}".format(np.sum(list_nodes_number), np.sum(list_connection_number)))

Total nodes:381 Total connections::4366
